In [1]:
import pandas as pd
import pickle

with open("afp_cls_30.pkl", "rb") as f:
    kmeans = pickle.load(f)

In [18]:
import tensorflow as tf
import tensorflow_hub as hub

class Word2VecVectorizer:
    def __init__(self):
        pass

    def fit(self, X, y):
        return self

    def transform(self, X):
        if isinstance(X, pd.core.series.Series):
            X = X.tolist()        
        return self.texts_encoder(X)

    def texts_encoder(self, texts):
        with tf.Graph().as_default():
            embed = hub.Module("https://tfhub.dev/google/nnlm-ja-dim128/1")
            embeddings = embed(texts)
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                sess.run(tf.tables_initializer())
                result = sess.run(embeddings)
        return result

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
df = pd.read_csv("credibility_dataset.csv")

In [5]:
df = df[['body', 'title', 'url', 'label']]

In [15]:
def get_sent_and_title(d):
    return d[1]['title'] + d[1]['body'].split("。")[0]

df['data'] = list(map(get_sent_and_title, df.iterrows()))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [19]:
X = Word2VecVectorizer().transform(df['data'])

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings


In [20]:
df['cluster'] = kmeans.predict(X)

In [23]:
import itertools
from nltk.corpus import stopwords
from collections import defaultdict
from tqdm import tqdm_notebook

def get_japanese_stopwords():
    stps = stopwords.words('japanese')
    return stps + [''.join(list(stp)) for stp in list(itertools.product(stps, stps))]

def word_counter(titles, stps, table=defaultdict(int)):
    for i, d in titles:
        tmp = d['data'].replace("\n","").split()
        for t in tmp:
            if t not in stps:
                table[t] += 1
    return table

stps = get_japanese_stopwords()
word_tables = []
for i in tqdm_notebook(range(30)):
    counter = 3
    word_table = word_counter(df[df['cluster'] == i].iterrows(), stps, defaultdict(int))
    word_tables.append(word_table.items())

In [24]:
for i in range(30):
    print('[cluster {}] '.format(i) + ' '.join([x[0] for x in sorted(word_tables[i], key=lambda x: x[1], reverse=True)[:20]]))

[cluster 0] ＣＮＮ 米 デモ 銃 大統領 ワシントン トランプ 移民 抗議 前 規制 米国 大使館 不法 拘束 首都 州 乱射 求める 解放
[cluster 1] 金 正 恩 北朝鮮 会談 委員 長 朝鮮 ＣＮＮ 労働党 氏 キム ジョン ウン 大統領 米 韓国 首脳 トランプ 朝
[cluster 2] シリア 米 ＣＮＮ 軍 ロシア 攻撃 化学 兵器 トランプ 元 英 東 大統領 グータ 派 氏 政権 スパイ 政府 首都
[cluster 3] ！ - : / 開催 展 音楽 . 』 かっぱ 時 ： 中 区 ハウス 銚子 com 〜 『 ”
[cluster 4] 米 ＣＮＮ 軍 機 中国 ミサイル ワシントン 北朝鮮 演習 ロシア 戦闘 軍事 発表 飛行 明らか 南シナ海 シリア 国防総省 海軍 実験
[cluster 5] 米 北朝鮮 ＣＮＮ 大統領 会談 トランプ 首脳 ワシントン 米国 / 長官 イラン 朝 核 中国 化 非核 的 ロイター 韓国
[cluster 6] 米 ＣＮＮ 州 墜落 事故 旅客機 死亡 機 空港 乗客 航空 中 旅客 着陸 国際 飛行 時 ワシントン アムトラック 列車
[cluster 7] 氏 トランプ 大統領 米 候補 ＣＮＮ 共和党 州 ワシントン 上院 民主党 選 支持 表明 補選 選挙 ロシア 議員 アラバマ わいせつ
[cluster 8] トランプ 米 大統領 氏 ＣＮＮ ワシントン ロシア 捜査 疑惑 特別 長官 選 米国 分かっ 発言 検察官 官 介入 批判 前
[cluster 9] / 円 June , 2018 : 分 読む ago ［ ］ - ロイター days 市場 東京 ビジネス AM ドル 米
[cluster 10] / ロイター June , 2018 : ago 分 読む ［ ］ - ％ ビジネス AM days ． . 東京 発表
[cluster 11] 米 大統領 トランプ 会談 ＣＮＮ ワシントン / ロシア 首脳 長官 官 氏 米国 発表 補佐 朝 ワールド June , 2018
[cluster 12] / 米 ロイター June , 2018 : 分 読む ［ ］ - ago . ( ) days AM 億 ドル
[cluster 13] ＣＮＮ 米 元 大統

In [25]:
for i in range(30):
    print("[{}]".format(i))
    counter = 3
    for j, d in df[df['cluster'] == i].iterrows():
        print(d['data'])
        counter -= 1
        if counter == 0:
            break
    print("")

[0]
米 セブン イレブン の 不法 移民 雇用 、 当局 が 全米 で 一斉 摘発 
ニューヨーク （ ＣＮＮＭｏｎｅｙ ）   米 移民 税関 捜査 局 （ ＩＣ Ｅ ） は １ ０ 日 、 全米 の セブン イレブン で 不法 移民 雇用 の 一斉 摘発 を 実施 し た 
英 女王 の 誕生 祝賀 パレード 、 ターバン 姿 の 兵士 に 注目 集まる 
（ ＣＮＮ ）   英 ロンドン で このほど 行わ れ た エリザベス 女王 の ９ ２ 回 目 の 誕生 日 を 祝う 英軍 の パレード に １ ０ ０ ０ 人 を 超える 兵士 が 参加 する なか 、 ひときわ 注目 を 集め た 人物 が い た 
カタルーニャ 州 の 前 首相 、 ドイツ で 拘束 独立 派 は 抗議 デモ 
（ ＣＮＮ ）   スペイン ・ カタルーニャ 州 の 首相 として 独立 運動 を 率い て い た プッチダモン 氏 が ２ ５ 日 、 ドイツ で 身柄 を 拘束 さ れ た 

[1]
米 朝 首脳 会談 開催 へ 、 北朝鮮 で は どの よう に 報じ られ た か 
（ ＣＮＮ ）   北朝鮮 の 人々 は １ １ 日 、 同国 の 最高 指導 者 、 金 正 恩 （ キム ジョン ウン ） 朝鮮 労働党 委員 長 が 米国 の ドナルド ・ トランプ 大統領 と の 歴史 的 な 首脳 会談 に 向け 、 シンガポール に 到着 し た という ニュース で 目覚め た 
金 正 恩 氏 「 核 実験 の 必要 なくなっ た 」 弾道 ミサイル 試射 も 中止 
ソウル （ ＣＮＮ ）   北朝鮮 の 朝鮮中央通信 （ ＫＣＮＡ ） は ２ １ 日 、 金 正 恩 （ キム ジョン ウン ） 朝鮮 労働党 委員 長 が 核 実験 や 大陸 間 弾道 ミサイル の 試射 は 必要 なくなっ た と の 認識 を 示し た と 伝え た 
李 雪 主 氏 に 「 ファーストレディー 」 の 肩書 、 権力 構造 に 変化 か 
（ ＣＮＮ ）   北朝鮮 の 国営 メディア が このほど 、 金 正 恩 （ キム ジョン ウン ） 朝鮮 労働党 委員 長 の 李 雪 主 （ リソルジュ ） 夫人 について 「 尊敬 する 李 雪 主 女史 」 と 伝え た

In [28]:
total = []
rate = []
for i in range(30):
    total.append(sum(df[df['cluster'] == i]['label']))
    rate.append(total[i]/df[df['cluster'] == i].shape[0])

In [30]:
for i, r in enumerate(rate):
    print("[{}]:{}%".format(i, int(r*100)))

[0]:3%
[1]:1%
[2]:0%
[3]:76%
[4]:10%
[5]:0%
[6]:4%
[7]:0%
[8]:9%
[9]:2%
[10]:1%
[11]:0%
[12]:9%
[13]:4%
[14]:74%
[15]:21%
[16]:0%
[17]:37%
[18]:5%
[19]:15%
[20]:0%
[21]:77%
[22]:0%
[23]:87%
[24]:57%
[25]:11%
[26]:18%
[27]:30%
[28]:12%
[29]:13%


In [31]:
import MeCab
sent = "トランプ氏、各国政府に欠席写真シール配布　存在感アピール トランプ米大統領が、自身の写真を楕円形にくりぬいた「欠席シール」を各国政府や国際機関に配布していたことが、２日までにわかった。"
tagger = MeCab.Tagger("-Owakati")
sent = tagger.parse(sent)
x = Word2VecVectorizer().transform([sent])
kmeans.predict(x)

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings


array([23], dtype=int32)